# Notebook 2 of 2

## This noteboook continues the work started in Notebook 1.        
> ### Here we will iterate through several neural network models to find the best accurracy in predicting whether a B-cell is normal or malignant with Acute Lymphoblastic Leukemia


In [1]:
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning)
warnings.filterwarnings(action='ignore', category=DeprecationWarning)
import pandas as pd
import numpy as np
import csv
import os
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import StandardScaler, LabelBinarizer, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_predict
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import load_model
from keras import regularizers
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras import models
from keras import layers
from keras import optimizers
from keras import initializers
from keras.wrappers.scikit_learn import KerasRegressor
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Dense, Activation, Dropout, Conv2D, MaxPooling2D,BatchNormalization, Flatten
from tensorflow.keras.optimizers import Adam, Adamax, RMSprop
from tensorflow.keras.metrics import binary_crossentropy, categorical_crossentropy
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model, load_model, Sequential
import numpy as np
import pandas as pd
import shutil
import time
import cv2 as cv2
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight 
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import seaborn as sns
sns.set_style('darkgrid')
from PIL import Image
import time
import scipy
from scipy import ndimage
from sklearn.metrics import confusion_matrix, classification_report
from IPython.core.display import display, HTML
# stop annoying tensorflow warning messages
import logging
import splitfolders
logging.getLogger("tensorflow").setLevel(logging.ERROR)
np.random.seed(123)
print ('modules loaded')

modules loaded


### First we must generate the data in this notebook.

In [2]:
# Create data generators
train_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
        'data/tvt/train',
        class_mode='binary',
        target_size=(225, 225), batch_size=10024)

val_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
        'data/tvt/validation',
        class_mode='binary',
        target_size=(225, 225), batch_size = 2379)

test_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
        'data/tvt/test',
        class_mode='binary',
        target_size=(225, 225), batch_size = 125)

Found 10024 images belonging to 2 classes.
Found 2379 images belonging to 2 classes.
Found 125 images belonging to 2 classes.


In [3]:
# # create the data sets
# train_images, train_labels = next(train_generator)
# val_images, val_labels = next(val_generator)
# test_images, test_labels = next(test_generator)

In [4]:
# #  Un-row the image matrices for modeling
# train_x = train_images.reshape(train_images.shape[0], -1)
# test_x = test_images.reshape(test_images.shape[0], -1)
# val_x = val_images.reshape(val_images.shape[0], -1)

In [5]:
# The datasets is imbalanced so we will define class_weight = balanced
class_weights = class_weight.compute_class_weight(class_weight='balanced',
                                                  classes=np.unique(train_generator.classes),
                                                  y=train_generator.classes
                                                 )

train_class_weights = dict(enumerate(class_weights))

## Build a basic Convolutional Neural Network

In [6]:
# Instantiate the model and build it's layers
cnn1 = models.Sequential()
cnn1.add(layers.Conv2D(16,(3,3), activation='relu', input_shape=(225,225,3)))
cnn1.add(layers.MaxPool2D(2,2))
cnn1.add(layers.Conv2D(32,(3,3), activation='relu'))
cnn1.add(layers.MaxPool2D(2,2))
cnn1.add(layers.Conv2D(64,(3,3), activation='relu'))
cnn1.add(layers.MaxPool2D(2,2))
cnn1.add(layers.Flatten())
cnn1.add(layers.Dense(512, activation='relu'))
cnn1.add(layers.Dense(1, activation='sigmoid'))

2022-03-01 09:48:05.357362: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
# Compile the model
cnn1.compile(optimizer=RMSprop(learning_rate=0.001),
            loss='binary_crossentropy',
            metrics=['accuracy'])

# Fit and Train the model
cnn1_history = cnn1.fit(train_generator,
                        steps_per_epoch=20,
                        epochs=50,
                        batch_size=500,
                        validation_data=(val_generator),
                        class_weight=train_class_weights,)

Epoch 1/50


InvalidArgumentError: Graph execution error:

Detected at node 'gradient_tape/sequential/conv2d_1/BiasAdd/BiasAddGrad' defined at (most recent call last):
    File "/Users/patrick/opt/anaconda3/lib/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/Users/patrick/opt/anaconda3/lib/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/Users/patrick/opt/anaconda3/lib/python3.9/site-packages/ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "/Users/patrick/opt/anaconda3/lib/python3.9/site-packages/traitlets/config/application.py", line 846, in launch_instance
      app.start()
    File "/Users/patrick/opt/anaconda3/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 677, in start
      self.io_loop.start()
    File "/Users/patrick/opt/anaconda3/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()

In [ ]:
def visualize_training_results(results):
    history = results.history
    plt.figure()
    plt.plot(history['val_loss'])
    plt.plot(history['loss'])
    plt.legend(['val_loss', 'loss'])
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.show()
    
    plt.figure()
    plt.plot(history['val_acc'])
    plt.plot(history['acc'])
    plt.legend(['val_acc', 'acc'])
    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.show()

visualize_training_results(cnn1_history)